# CAMO-Net - Análise Exploratória Inicial
## Projeto: Resistência Antimicrobiana (AMR)

**Objetivo:** Realizar EDA automática dos dados brutos da carga inicial

**Data:** 2025-11-26

**Autor:** Engenheiro de Dados Sênior

## 1. Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 2. Função de Leitura com Fallback de Encoding

In [2]:
def read_csv_with_fallback(filepath):
    """
    Tenta ler CSV com diferentes encodings.
    Ordem: utf-8 -> latin1 -> cp1252
    """
    encodings = ['utf-8', 'latin1', 'cp1252']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(filepath, encoding=encoding, sep=';', low_memory=False)
            print(f"✓ Arquivo lido com sucesso usando encoding: {encoding}")
            return df, encoding
        except UnicodeDecodeError:
            continue
        except Exception as e:
            # Tenta com vírgula como separador
            try:
                df = pd.read_csv(filepath, encoding=encoding, sep=',', low_memory=False)
                print(f"✓ Arquivo lido com sucesso usando encoding: {encoding} e separador ','")
                return df, encoding
            except:
                continue
    
    raise ValueError(f"Não foi possível ler o arquivo {filepath} com nenhum encoding testado.")

## 3. Carregamento dos Dados

In [3]:
# Caminho da pasta de dados brutos
data_path = Path('../..') / 'data' / '20250101_carga_inicial_ana_roccio'

# Listar todos os arquivos CSV
csv_files = list(data_path.glob('*.csv'))
print(f"Total de arquivos CSV encontrados: {len(csv_files)}\n")

for file in csv_files:
    print(f"  - {file.name}")

Total de arquivos CSV encontrados: 9

  - TAB_ATENDIMENTO.csv
  - TAB_MEDICAMENTO.csv
  - TAB_ATENDIMENTO_ANALISE.csv
  - TAB_ATEND_DIAG.csv
  - TAB_CID_DIAGNOSTICO.csv
  - TAB_UNIDADE_SAUDE.csv
  - TAB_MEDPRESCRITO_ANALISE.csv
  - TAB_CIAP_DIAGNOSTICO.csv
  - TAB_MED_PRESCRITO.csv


In [4]:
# Dicionário para armazenar os dataframes
dataframes = {}
encodings_used = {}

# Carregar cada arquivo
for csv_file in csv_files:
    table_name = csv_file.stem  # Nome sem extensão
    print(f"\n{'='*80}")
    print(f"Carregando: {table_name}")
    print(f"{'='*80}")
    
    df, encoding = read_csv_with_fallback(csv_file)
    dataframes[table_name] = df
    encodings_used[table_name] = encoding
    
    print(f"Shape: {df.shape}")
    print(f"Encoding usado: {encoding}")


Carregando: TAB_ATENDIMENTO
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (480439, 1)
Encoding usado: utf-8

Carregando: TAB_MEDICAMENTO
✓ Arquivo lido com sucesso usando encoding: utf-8 e separador ','
Shape: (33246, 11)
Encoding usado: utf-8

Carregando: TAB_ATENDIMENTO_ANALISE
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (298848, 1)
Encoding usado: utf-8

Carregando: TAB_ATEND_DIAG
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (574676, 1)
Encoding usado: utf-8

Carregando: TAB_CID_DIAGNOSTICO
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (896, 1)
Encoding usado: utf-8

Carregando: TAB_UNIDADE_SAUDE
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (50, 1)
Encoding usado: utf-8

Carregando: TAB_MEDPRESCRITO_ANALISE
✓ Arquivo lido com sucesso usando encoding: utf-8 e separador ','
Shape: (269123, 8)
Encoding usado: utf-8

Carregando: TAB_CIAP_DIAGNOSTICO
✓ Arquivo lido com sucesso usando encoding: utf-8
Shape: (726, 1)
Encoding usad

## 4. Análise Detalhada de Cada Tabela

In [5]:
def analyze_dataframe(df, name):
    """
    Análise completa de um dataframe
    """
    print(f"\n{'#'*80}")
    print(f"# ANÁLISE: {name}")
    print(f"{'#'*80}\n")
    
    # Info básica
    print("\n[1] INFORMAÇÕES GERAIS")
    print("-" * 80)
    print(df.info())
    
    # Primeiras linhas
    print("\n[2] PRIMEIRAS 5 LINHAS")
    print("-" * 80)
    display(df.head())
    
    # Valores nulos
    print("\n[3] CONTAGEM DE VALORES NULOS")
    print("-" * 80)
    null_counts = df.isnull().sum()
    null_pct = (null_counts / len(df) * 100).round(2)
    null_summary = pd.DataFrame({
        'Nulos': null_counts,
        'Percentual (%)': null_pct
    }).sort_values('Nulos', ascending=False)
    
    # Mostrar apenas colunas com nulos
    null_summary_filtered = null_summary[null_summary['Nulos'] > 0]
    if len(null_summary_filtered) > 0:
        display(null_summary_filtered)
    else:
        print("✓ Nenhum valor nulo encontrado!")
    
    # Duplicatas
    print("\n[4] ANÁLISE DE DUPLICATAS")
    print("-" * 80)
    duplicates = df.duplicated().sum()
    print(f"Total de linhas duplicadas: {duplicates}")
    print(f"Percentual de duplicatas: {(duplicates/len(df)*100):.2f}%")
    
    # Estatísticas descritivas para colunas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("\n[5] ESTATÍSTICAS DESCRITIVAS (Colunas Numéricas)")
        print("-" * 80)
        display(df[numeric_cols].describe())
    
    print("\n" + "="*80 + "\n")

In [6]:
# Analisar cada tabela
for table_name, df in dataframes.items():
    analyze_dataframe(df, table_name)


################################################################################
# ANÁLISE: TAB_ATENDIMENTO
################################################################################


[1] INFORMAÇÕES GERAIS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480439 entries, 0 to 480438
Data columns (total 1 columns):
 #   Column                                                                                          Non-Null Count   Dtype 
---  ------                                                                                          --------------   ----- 
 0   COD_ATENDIMENTO,COD_PACIENTE,COD_UNIDADE_SAUDE,DATA_ATENDIMENTO,ESPECIALIDADE,PERIODO_EXTRACAO  480439 non-null  object
dtypes: object(1)
memory usage: 3.7+ MB
None

[2] PRIMEIRAS 5 LINHAS
--------------------------------------------------------------------------------


,"COD_ATENDIMENTO,COD_PACIENTE,COD_UNIDADE_SAUDE,DATA_ATENDIMENTO,ESPECIALIDADE,PERIODO_EXTRACAO"
0,"00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,51131..."
1,"0000398021ccd3a488fa1a63dee3b2f0d471f9fd,63731..."
2,"0000a0d034eb5893f9d50fc4c038fac22e5c8e80,10583..."
3,"0000b4af921db374095a9e7ab87ee314522e4fe4,4708,..."
4,"0000fe5ae46d696f27604fc70946b290277bff95,6810,..."



[3] CONTAGEM DE VALORES NULOS
--------------------------------------------------------------------------------
✓ Nenhum valor nulo encontrado!

[4] ANÁLISE DE DUPLICATAS
--------------------------------------------------------------------------------
Total de linhas duplicadas: 0
Percentual de duplicatas: 0.00%



################################################################################
# ANÁLISE: TAB_MEDICAMENTO
################################################################################


[1] INFORMAÇÕES GERAIS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33246 entries, 0 to 33245
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   COD_MEDICAMENTO       33246 non-null  int64  
 1   NOME_MEDICAMENTO      33246 non-null  object 
 2   COD_PRODUTO           1175 non-null   float64
 3   NOME_PRODUTO       

,COD_MEDICAMENTO,NOME_MEDICAMENTO,COD_PRODUTO,NOME_PRODUTO,COMPOSTO_QUIMICO,E_ANTIBIOTICO,TIPO_USO,CONCENTRACAO,UNIDADE_APRESENTACAO,DESCRICAO_AAAA,DESCRICAO_AAAAA
0,49398,"ENALAPRIL, MALEATO 10 MG COMP.",2956.0,"ENALAPRIL, MALEATO 10 MG COMP",ENALAPRIL MALEATE,0,NaN,1 MG,COMPRIMIDO COM 10MG,NaN,NaN
1,49399,HIDROCLOROTIAZIDA 25 MG COMP.,2784.0,HIDROCLOROTIAZIDA 1MG/ML 50ML,HYDROCHLOROTHIAZIDE,0,NaN,1 MG,COMPRIMIDO C/ 25MG,NaN,NaN
2,49400,SINVASTATINA 20 MG COMP.,3002.0,SINVASTATINA 20 MG COMP...,SIMVASTATIN,0,NaN,1 MG,COMPRIMINDO COM 20MG,NaN,NaN
3,49401,CIPROFIBRATO 100MGG,1906.0,CIPROFIBRATO 100MG.,CIPROFIBRATE,0,NaN,1 MG,COMPRIMIDO COM 100MG,NaN,NaN
4,49406,SINVASTATINA 20 MG COMP.,3003.0,SINVASTATINA 20 MG COMP..,SIMVASTATIN,0,NaN,1 MG,MILIGRAMA,NaN,NaN



[3] CONTAGEM DE VALORES NULOS
--------------------------------------------------------------------------------


...

,"COD_CIAP,DIAG_ORIGINAL,DIAG_AGRUPADO,E_GUIA_ANALISE,DIAG_ANALISE,E_INFECCAO"
0,"-30,""Medical examination/health evaluation-com..."
1,"-31,""Medical examination/health evaluation-par..."
2,"-32,""Sensitivity test"",""Sensitivity test"",0,,0"
3,"-33,""Microbiological/immunological test"",""Micr..."
4,"-34,""Blood test"",""Blood test"",0,,0"



[3] CONTAGEM DE VALORES NULOS
--------------------------------------------------------------------------------
✓ Nenhum valor nulo encontrado!

[4] ANÁLISE DE DUPLICATAS
--------------------------------------------------------------------------------
Total de linhas duplicadas: 0
Percentual de duplicatas: 0.00%



################################################################################
# ANÁLISE: TAB_MED_PRESCRITO
################################################################################


[1] INFORMAÇÕES GERAIS
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443454 entries, 0 to 443453
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   COD_ATENDIMENTO   443454 non-null  object
 1   COD_MEDICAMENTO   443454 non-null  int64 
 2   QUANTIDADE        443454 non-null  int64 
 3   QTD_RECEITA       443454 non-null  i

,COD_ATENDIMENTO,COD_MEDICAMENTO,QUANTIDADE,QTD_RECEITA,DURACAO,PERIODO_EXTRACAO
0,0000b4af921db374095a9e7ab87ee314522e4fe4,57466,60,1,Tomar um comprimido de 12 em 12 hoRas,JAN23-SET23
1,0000b4af921db374095a9e7ab87ee314522e4fe4,57723,60,1,TOMAR 1CP 30 MINUTOS ANTES DO ALMOÇO E DO JANTAR,JAN23-SET23
2,0000b4af921db374095a9e7ab87ee314522e4fe4,57862,30,1,Tomar 1cp VO ao dia.,JAN23-SET23
3,0000b4af921db374095a9e7ab87ee314522e4fe4,447509,30,1,1CP 1X/SEMANA POR 6 MESSE,JAN23-SET23
4,00019328d3a64a85b98cc0c18b03c1ca2044678a,198400,30,1,TOMAR 1 CP 30 MIN ANTES DO CAFÉ,JAN23-SET23



[3] CONTAGEM DE VALORES NULOS
--------------------------------------------------------------------------------
✓ Nenhum valor nulo encontrado!

[4] ANÁLISE DE DUPLICATAS
--------------------------------------------------------------------------------
Total de linhas duplicadas: 0
Percentual de duplicatas: 0.00%

[5] ESTATÍSTICAS DESCRITIVAS (Colunas Numéricas)
--------------------------------------------------------------------------------


,COD_MEDICAMENTO,QUANTIDADE,QTD_RECEITA
count,443454.000000,4.434540e+05,443454.000000
mean,174632.194638,6.341080e+01,1.293821
std,211551.705169,1.668562e+04,0.924182
min,49398.000000,1.000000e+00,1.000000
25%,57490.000000,5.000000e+00,1.000000
50%,57702.000000,3.000000e+01,1.000000
75%,186871.000000,6.000000e+01,1.000000
max,840283.000000,1.111111e+07,12.000000


## 5. Análise de Chaves Comuns Entre Tabelas

In [7]:
print("ANÁLISE DE COLUNAS COMUNS ENTRE TABELAS")
print("="*80)

# Listar todas as colunas de cada tabela
all_columns = {}
for table_name, df in dataframes.items():
    all_columns[table_name] = set(df.columns)
    print(f"\n{table_name}:")
    print(f"  Colunas: {list(df.columns)}")

ANÁLISE DE COLUNAS COMUNS ENTRE TABELAS

TAB_ATENDIMENTO:
  Colunas: ['COD_ATENDIMENTO,COD_PACIENTE,COD_UNIDADE_SAUDE,DATA_ATENDIMENTO,ESPECIALIDADE,PERIODO_EXTRACAO']

TAB_MEDICAMENTO:
  Colunas: ['COD_MEDICAMENTO', 'NOME_MEDICAMENTO', 'COD_PRODUTO', 'NOME_PRODUTO', 'COMPOSTO_QUIMICO', 'E_ANTIBIOTICO', 'TIPO_USO', 'CONCENTRACAO', 'UNIDADE_APRESENTACAO', 'DESCRICAO_AAAA', 'DESCRICAO_AAAAA']

TAB_ATENDIMENTO_ANALISE:
  Colunas: ['COD_ATENDIMENTO,COD_PACIENTE,DATA_ATENDIMENTO,SEXO,IDADE,ESPECIALIDADE,COD_CID_CIAP,DIAGNOSTICAR_POR,DIAG_AGRUPADO,DIAG_ANALISE,E_DIAG_INFECCIOSO,PERIODO_EXTRACAO']

TAB_ATEND_DIAG:
  Colunas: ['COD_ATENDIMENTO,COD_CID,COD_CIAP,QUEIXA,DIAGNOSTICAR_POR']

TAB_CID_DIAGNOSTICO:
  Colunas: ['COD_CID,E_BASE,COD_CID_BASE,DIAG_ORIGINAL,DIAG_AGRUPADO,E_GUIA_ANALISE,DIAG_ANALISE,E_INFECCAO']

TAB_UNIDADE_SAUDE:
  Colunas: ['COD_UNIDADE_SAUDE,NOME_UNIDADE,TIPO,E_ANALIZADA']

TAB_MEDPRESCRITO_ANALISE:
  Colunas: ['COD_ATENDIMENTO', 'COD_MEDICAMENTO', 'NOME_MEDICAMENTO', '

In [8]:
# Identificar colunas comuns
print("\n" + "="*80)
print("COLUNAS POTENCIAIS PARA JOINS")
print("="*80)

# Palavras-chave para identificar possíveis chaves
key_patterns = ['id', 'cod', 'codigo', 'paciente', 'atendimento', 'data', 'medicamento', 'diagnostico']

for table_name, columns in all_columns.items():
    print(f"\n{table_name}:")
    potential_keys = [col for col in columns if any(pattern in col.lower() for pattern in key_patterns)]
    print(f"  Possíveis chaves: {potential_keys}")


COLUNAS POTENCIAIS PARA JOINS

TAB_ATENDIMENTO:
  Possíveis chaves: ['COD_ATENDIMENTO,COD_PACIENTE,COD_UNIDADE_SAUDE,DATA_ATENDIMENTO,ESPECIALIDADE,PERIODO_EXTRACAO']

TAB_MEDICAMENTO:
  Possíveis chaves: ['COD_PRODUTO', 'UNIDADE_APRESENTACAO', 'COD_MEDICAMENTO', 'NOME_MEDICAMENTO']

TAB_ATENDIMENTO_ANALISE:
  Possíveis chaves: ['COD_ATENDIMENTO,COD_PACIENTE,DATA_ATENDIMENTO,SEXO,IDADE,ESPECIALIDADE,COD_CID_CIAP,DIAGNOSTICAR_POR,DIAG_AGRUPADO,DIAG_ANALISE,E_DIAG_INFECCIOSO,PERIODO_EXTRACAO']

TAB_ATEND_DIAG:
  Possíveis chaves: ['COD_ATENDIMENTO,COD_CID,COD_CIAP,QUEIXA,DIAGNOSTICAR_POR']

TAB_CID_DIAGNOSTICO:
  Possíveis chaves: ['COD_CID,E_BASE,COD_CID_BASE,DIAG_ORIGINAL,DIAG_AGRUPADO,E_GUIA_ANALISE,DIAG_ANALISE,E_INFECCAO']

TAB_UNIDADE_SAUDE:
  Possíveis chaves: ['COD_UNIDADE_SAUDE,NOME_UNIDADE,TIPO,E_ANALIZADA']

TAB_MEDPRESCRITO_ANALISE:
  Possíveis chaves: ['UNIDADE_APRESENTACAO', 'COD_MEDICAMENTO', 'NOME_MEDICAMENTO', 'COD_ATENDIMENTO']

TAB_CIAP_DIAGNOSTICO:
  Possíveis chaves

In [9]:
# Análise específica: Consultas/Diagnósticos vs Medicamentos
print("\n" + "="*80)
print("ANÁLISE DE RELACIONAMENTO: DIAGNÓSTICOS ↔ MEDICAMENTOS")
print("="*80)

# Identificar tabelas relevantes
diag_tables = [name for name in dataframes.keys() if 'DIAG' in name or 'ATEND' in name]
med_tables = [name for name in dataframes.keys() if 'MED' in name]

print(f"\nTabelas de Diagnóstico/Atendimento: {diag_tables}")
print(f"Tabelas de Medicamentos: {med_tables}")

# Encontrar colunas em comum
if diag_tables and med_tables:
    for diag_table in diag_tables:
        for med_table in med_tables:
            common_cols = all_columns[diag_table].intersection(all_columns[med_table])
            if common_cols:
                print(f"\n{diag_table} ∩ {med_table}:")
                print(f"  Colunas comuns: {list(common_cols)}")


ANÁLISE DE RELACIONAMENTO: DIAGNÓSTICOS ↔ MEDICAMENTOS

Tabelas de Diagnóstico/Atendimento: ['TAB_ATENDIMENTO', 'TAB_ATENDIMENTO_ANALISE', 'TAB_ATEND_DIAG', 'TAB_CID_DIAGNOSTICO', 'TAB_CIAP_DIAGNOSTICO']
Tabelas de Medicamentos: ['TAB_MEDICAMENTO', 'TAB_MEDPRESCRITO_ANALISE', 'TAB_MED_PRESCRITO']


## 6. Investigação de Negócio (AMR)

### 6.1 Top 20 Diagnósticos (CID) Mais Comuns

In [10]:
# Identificar tabelas com diagnósticos CID
cid_tables = [name for name in dataframes.keys() if 'CID' in name]
print(f"Tabelas com CID: {cid_tables}\n")

for table_name in cid_tables:
    df = dataframes[table_name]
    print(f"\n{'='*80}")
    print(f"ANÁLISE CID - {table_name}")
    print(f"{'='*80}")
    
    # Procurar colunas que possam conter CID
    cid_columns = [col for col in df.columns if 'cid' in col.lower() or 'diag' in col.lower()]
    print(f"Colunas relacionadas a CID: {cid_columns}\n")
    
    # Analisar cada coluna
    for col in cid_columns:
        if df[col].notna().sum() > 0:
            print(f"\nTop 20 - {col}:")
            top_20 = df[col].value_counts().head(20)
            display(pd.DataFrame({
                'Diagnóstico': top_20.index,
                'Frequência': top_20.values,
                'Percentual (%)': (top_20.values / top_20.sum() * 100).round(2)
            }).reset_index(drop=True))

Tabelas com CID: ['TAB_CID_DIAGNOSTICO']


ANÁLISE CID - TAB_CID_DIAGNOSTICO
Colunas relacionadas a CID: ['COD_CID,E_BASE,COD_CID_BASE,DIAG_ORIGINAL,DIAG_AGRUPADO,E_GUIA_ANALISE,DIAG_ANALISE,E_INFECCAO']


Top 20 - COD_CID,E_BASE,COD_CID_BASE,DIAG_ORIGINAL,DIAG_AGRUPADO,E_GUIA_ANALISE,DIAG_ANALISE,E_INFECCAO:


,Diagnóstico,Frequência,Percentual (%)
0,"A00,1,A00,""Intestinal infectious diseases"",""In...",1,5.0
1,"A01,1,A01,""Intestinal infectious diseases"",""In...",1,5.0
2,"M10,0,M10,Gout,Gout,0,,0",1,5.0
3,"M139,0,M13,""Enthesopathies,unspecified"",""Enthe...",1,5.0
4,"M150,0,M15,""Primary generalized (osteo),arthri...",1,5.0
5,"M16,0,M16,""Coxarthrosis [arthrosis of hip]"",""C...",1,5.0
6,"M160,0,M16,Bunion,Bunion,0,,0",1,5.0
7,"M169,0,M16,""Other chondroplasia"",""Other chondr...",1,5.0
8,"M17,0,M17,""Gonarthrosis [osteoarthritis] (bila...",1,5.0
9,"M170,0,M17,""Unilateral primary osteoarthritis ...",1,5.0


### 6.2 Top 20 Medicamentos Mais Prescritos

In [11]:
# Identificar tabelas com medicamentos
med_tables = [name for name in dataframes.keys() if 'MED' in name]
print(f"Tabelas com Medicamentos: {med_tables}\n")

for table_name in med_tables:
    df = dataframes[table_name]
    print(f"\n{'='*80}")
    print(f"ANÁLISE MEDICAMENTOS - {table_name}")
    print(f"{'='*80}")
    
    # Procurar colunas que possam conter nomes de medicamentos
    med_columns = [col for col in df.columns if any(keyword in col.lower() 
                   for keyword in ['medicamento', 'med', 'nome', 'principio', 'substancia'])]
    print(f"Colunas relacionadas a Medicamentos: {med_columns}\n")
    
    # Analisar cada coluna
    for col in med_columns:
        if df[col].notna().sum() > 0:
            print(f"\nTop 20 - {col}:")
            top_20 = df[col].value_counts().head(20)
            display(pd.DataFrame({
                'Medicamento': top_20.index,
                'Frequência': top_20.values,
                'Percentual (%)': (top_20.values / top_20.sum() * 100).round(2)
            }).reset_index(drop=True))

Tabelas com Medicamentos: ['TAB_MEDICAMENTO', 'TAB_MEDPRESCRITO_ANALISE', 'TAB_MED_PRESCRITO']


ANÁLISE MEDICAMENTOS - TAB_MEDICAMENTO
Colunas relacionadas a Medicamentos: ['COD_MEDICAMENTO', 'NOME_MEDICAMENTO', 'NOME_PRODUTO']


Top 20 - COD_MEDICAMENTO:


,Medicamento,Frequência,Percentual (%)
0,49398,1,5.0
1,718100,1,5.0
2,718421,1,5.0
3,718420,1,5.0
4,718416,1,5.0
5,718408,1,5.0
6,718391,1,5.0
7,718385,1,5.0
8,718382,1,5.0
9,718378,1,5.0



Top 20 - NOME_MEDICAMENTO:


,Medicamento,Frequência,Percentual (%)
0,COLECALCIFEROL,259,16.61
1,COLECALCIFEROL,184,11.80
2,VITAMINA D,92,5.90
3,ATORVASTATINA,83,5.32
4,VENLAFAXINA,73,4.68
5,ESCITALOPRAM,71,4.55
6,ROSUVASTATINA,68,4.36
7,GLIFAGE XR,67,4.30
8,VITAMINA D,66,4.23
9,VENLAFAXINA,61,3.91



Top 20 - NOME_PRODUTO:


...

,Medicamento,Frequência,Percentual (%)
0,57466,13128,13.62
1,55448,9867,10.23
2,57467,8516,8.83
3,57462,8505,8.82
4,57468,6820,7.07
5,57472,6198,6.43
6,57503,4922,5.10
7,57464,4613,4.78
8,57640,4030,4.18
9,57504,3636,3.77



Top 20 - NOME_MEDICAMENTO:


,Medicamento,Frequência,Percentual (%)
0,LOSARTANA POTASSICA 50MG,13369,13.16
1,DIPIRONA SODICA 500MG,9872,9.72
2,HIDROCLOROTIAZIDA 25MG,8733,8.60
3,OMEPRAZOL 20MG,8621,8.49
4,SINVASTATINA 20MG,6993,6.89
5,SERTRALINA 50MG,6237,6.14
6,"ANLODIPINO, BESILATO 5MG",4998,4.92
7,ACIDO ACETILSALICILICO 100MG,4745,4.67
8,COLECALCIFEROL,4531,4.46
9,LORATADINA 10MG,4030,3.97



ANÁLISE MEDICAMENTOS - TAB_MED_PRESCRITO
Colunas relacionadas a Medicamentos: ['COD_MEDICAMENTO']


Top 20 - COD_MEDICAMENTO:


,Medicamento,Frequência,Percentual (%)
0,57466,15138,11.47
1,57472,13767,10.43
2,55448,12504,9.47
3,57462,10352,7.84
4,57467,9701,7.35
5,57490,7735,5.86
6,57468,7709,5.84
7,57574,6021,4.56
8,57464,6016,4.56
9,57503,5968,4.52


### 6.3 Investigação de Variações de Antibióticos

In [12]:
# Lista de antibióticos para investigar
antibiotics = ['amoxicilina', 'azitromicina']

print("INVESTIGAÇÃO DE VARIAÇÕES DE ESCRITA DE ANTIBIÓTICOS")
print("="*80)

for table_name in med_tables:
    df = dataframes[table_name]
    
    # Procurar colunas com nomes de medicamentos
    med_columns = [col for col in df.columns if any(keyword in col.lower() 
                   for keyword in ['medicamento', 'med', 'nome', 'principio', 'substancia'])]
    
    for col in med_columns:
        if df[col].notna().sum() > 0:
            print(f"\n{'='*80}")
            print(f"Tabela: {table_name} | Coluna: {col}")
            print(f"{'='*80}")
            
            for antibiotic in antibiotics:
                # Buscar variações (case-insensitive)
                mask = df[col].astype(str).str.lower().str.contains(antibiotic, na=False)
                variations = df[mask][col].value_counts()
                
                if len(variations) > 0:
                    print(f"\n🔍 Variações de '{antibiotic.upper()}':")
                    print("-" * 80)
                    display(pd.DataFrame({
                        'Variação': variations.index,
                        'Frequência': variations.values,
                        'Percentual (%)': (variations.values / variations.sum() * 100).round(2)
                    }).reset_index(drop=True))
                    print(f"\nTotal de variações encontradas: {len(variations)}")
                    print(f"Total de registros: {variations.sum()}")
                else:
                    print(f"\n⚠️  Nenhuma variação de '{antibiotic.upper()}' encontrada nesta coluna.")

INVESTIGAÇÃO DE VARIAÇÕES DE ESCRITA DE ANTIBIÓTICOS

Tabela: TAB_MEDICAMENTO | Coluna: COD_MEDICAMENTO

⚠️  Nenhuma variação de 'AMOXICILINA' encontrada nesta coluna.

⚠️  Nenhuma variação de 'AZITROMICINA' encontrada nesta coluna.

Tabela: TAB_MEDICAMENTO | Coluna: NOME_MEDICAMENTO

🔍 Variações de 'AMOXICILINA':
--------------------------------------------------------------------------------


,Variação,Frequência,Percentual (%)
0,AMOXICILINA + CLAVULANATO,9,22.5
1,AMOXICILINA + CLAVULANATO,3,7.5
2,AMOXICILINA,2,5.0
3,AMOXICILINA 500MG CAP,1,2.5
4,"AMOXICILINA 250MG + CLAVULANATO 62,5MG/5ML",1,2.5
5,AMOXICILINA + CLAVULANATO BD,1,2.5
6,PYLORIPAC (LANSOPRAZOL 30 MG + CLARITROMICINA ...,1,2.5
7,AMOXICILINA COM CLAVULANATO 875/125MG,1,2.5
8,"AMOXICILINA 250/ML + CLAVULANATO 62,5/ML",1,2.5
9,AMOXICILINA + CLAVULANATO 875 + 125 MG,1,2.5



Total de variações encontradas: 29
Total de registros: 40

🔍 Variações de 'AZITROMICINA':
--------------------------------------------------------------------------------


,Variação,Frequência,Percentual (%)
0,AZITROMICINA,5,50.0
1,AZITROMICINA 500MG,1,10.0
2,AZITROMICINA 600MG (200MG/5ML) SUSPENSAO ORAL ...,1,10.0
3,AZITROMICINA 500 MG,1,10.0
4,AZITROMICINA 500 MG,1,10.0
5,AZITROMICINA,1,10.0



Total de variações encontradas: 6
Total de registros: 10

Tabela: TAB_MEDICAMENTO | Coluna: NOME_PRODUTO

🔍 Variações de 'AMOXICILINA':
--------------------------------------------------------------------------------


...


Total de variações encontradas: 2
Total de registros: 2

Tabela: TAB_MEDPRESCRITO_ANALISE | Coluna: COD_MEDICAMENTO

⚠️  Nenhuma variação de 'AMOXICILINA' encontrada nesta coluna.

⚠️  Nenhuma variação de 'AZITROMICINA' encontrada nesta coluna.

Tabela: TAB_MEDPRESCRITO_ANALISE | Coluna: NOME_MEDICAMENTO

🔍 Variações de 'AMOXICILINA':
--------------------------------------------------------------------------------


,Variação,Frequência,Percentual (%)
0,AMOXICILINA 500MG CAP,1609,57.55
1,AMOXICILINA 500MG/CLAVULANATO POTASSIO 125MG CPR,917,32.80
2,AMOXICILINA 250MG/5ML PO P/ SUSP. 150ML,142,5.08
3,"AMOXICILINA 250 MG/ML CLAV.DE POT 62,5MG/ML PO...",73,2.61
4,AMOXICILINA + CLAVULANATO,14,0.50
5,AMOXICILINA 250MG/5ML PO P/ SUSP. 60ML,8,0.29
6,AMOXICILINA,7,0.25
7,AMOXICILINA + CLAVULANATO,4,0.14
8,AMOXICILINA 250MG/5ML,3,0.11
9,AMOXICILINA + CLAVULANATO DE POTÁSSIO,2,0.07



Total de variações encontradas: 27
Total de registros: 2796

🔍 Variações de 'AZITROMICINA':
--------------------------------------------------------------------------------


,Variação,Frequência,Percentual (%)
0,AZITROMICINA 500MG,1135,95.3
1,AZITROMICINA 600MG (200MG/5ML) SUSPENSAO ORAL ...,50,4.2
2,AZITROMICINA,6,0.5



Total de variações encontradas: 3
Total de registros: 1191

Tabela: TAB_MED_PRESCRITO | Coluna: COD_MEDICAMENTO

⚠️  Nenhuma variação de 'AMOXICILINA' encontrada nesta coluna.

⚠️  Nenhuma variação de 'AZITROMICINA' encontrada nesta coluna.


## 7. Resumo da Análise

In [13]:
print("\n" + "#"*80)
print("# RESUMO EXECUTIVO DA ANÁLISE")
print("#"*80 + "\n")

print("[1] TABELAS CARREGADAS")
print("-" * 80)
summary_data = []
for table_name, df in dataframes.items():
    summary_data.append({
        'Tabela': table_name,
        'Linhas': len(df),
        'Colunas': len(df.columns),
        'Encoding': encodings_used[table_name],
        'Duplicatas': df.duplicated().sum(),
        'Colunas com Nulos': df.isnull().any().sum()
    })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

print(f"\nTotal de registros no dataset: {summary_df['Linhas'].sum():,}")
print(f"Total de tabelas: {len(dataframes)}")


################################################################################
# RESUMO EXECUTIVO DA ANÁLISE
################################################################################

[1] TABELAS CARREGADAS
--------------------------------------------------------------------------------


,Tabela,Linhas,Colunas,Encoding,Duplicatas,Colunas com Nulos
0,TAB_ATENDIMENTO,480439,1,utf-8,0,0
1,TAB_MEDICAMENTO,33246,11,utf-8,0,6
2,TAB_ATENDIMENTO_ANALISE,298848,1,utf-8,0,0
3,TAB_ATEND_DIAG,574676,1,utf-8,0,0
4,TAB_CID_DIAGNOSTICO,896,1,utf-8,0,0
5,TAB_UNIDADE_SAUDE,50,1,utf-8,0,0
6,TAB_MEDPRESCRITO_ANALISE,269123,8,utf-8,0,1
7,TAB_CIAP_DIAGNOSTICO,726,1,utf-8,0,0
8,TAB_MED_PRESCRITO,443454,6,utf-8,0,0



Total de registros no dataset: 2,101,458
Total de tabelas: 9


## 8. Conclusões e Próximos Passos

### Principais Descobertas:
1. **Qualidade dos Dados**: [A ser preenchido após execução]
2. **Chaves de Relacionamento**: [A ser preenchido após execução]
3. **Diagnósticos Principais**: [A ser preenchido após execução]
4. **Medicamentos Mais Prescritos**: [A ser preenchido após execução]
5. **Variações de Antibióticos**: [A ser preenchido após execução]

### Próximos Passos:
1. Implementar pipeline `raw_to_bronze.py`
2. Implementar pipeline `bronze_to_silver.py`
3. Criar modelo dimensional para análise AMR
4. Desenvolver dashboards de monitoramento